# Importando pacotes

Primeiramente, é feita a importação dos pacotes que são necessários para a realização da atividade proposta. Sendo eles:

*   Pandas: para manipulação e análise de dados;
*   Numpy: para uso de funções matemáticas.

Um comentário: quase todos os _dataframes_ serão apresentados com auxílio do método `head`, que permite fazer a visualização das primeiras observações do _dataframe_. Isso é comentado aqui para não haver necessidade de repetição em cada instância que o método é utilizado.

In [ ]:
import pandas as pd
import numpy as np

# Importação dos dados

Começou-se importando os dados e fazendo a leitura da estrutura básica deles. É necessário passar o argumento `sep = ";"` devido à natureza do separador nos bancos de dados e, visto que não há nome para as colunas do banco `movies`, foi decidido usar os nomes `Movie_Id` e `Movie`.

Após isso foi feita a separação da coluna `Movie` em `Movie` e `Year`. A coluna `Movie` contém o título do filme, enquanto a coluna `Year` contém a data de lançamento do mesmo. Esse passo não é necessário, contudo a estrutura do banco se torna mais apresentável dessa forma. Para tanto, foram utilizadas funções que trabalham com _strings_:

*   `str.split` que divide uma _string_ em demais _strings_ de acordo com o separador determinado (nesse caso `","`);
*   `str.replace` que altera partes de uma _string_ de acordo com os argumentos `oldvalue` e `newvalue`, onde o primeiro é substituído pelo segundo.

PS: A leitura dos dados via `read_csv` está usando como caminho simplesmente o nome do arquivo `movies.csv`. O mesmo será feito para o arquivo `customers.csv`. Durante a execução dessa análise, contudo, esse não era o caminho, como o banco de dados não será provido com esse código, então não faria sentido manter o mesmo caminho que foi usado durante. Dessa forma, para a leitura apropriada do código, é necessário colocar esse arquivo .ipynb no mesmo diretório que os bancos de dados, ou alterar o caminho das funções `read_csv`.

In [ ]:
movies = pd.read_csv("movies.csv", sep = ";", names = ["Movie_Id", "Movie"]) # Lendo o arquivo movies.csv

movie_year = movies["Movie"].str.split(",", n = 1, expand = True) # Criando um df que separa filme e ano

movies["Movie"] = movie_year[0] # Alterando a coluna Movie para conter somente o título do filme
movies["Year"] = movie_year[1] # Criando a coluna Year que contém o ano de lançamento do filme

movies["Movie"] = movies["Movie"].str.replace("(", "")
movies["Year"] = movies["Year"].str.replace(")", "")

movies.head(5)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  if __name__ == '__main__':


,Movie_Id,Movie,Year
0,1,Dinosaur Planet,2003
1,2,Isle of Man TT 2004 Review,2004
2,3,Character,1997
3,4,Paula Abdul's Get Up & Dance,1994
4,5,The Rise and Fall of ECW,2004


Para o segundo _dataframe_, `customers`, apenas foi necessário passar o argumento `sep = ";"` pelo mesmo motivo que anteriormente.

In [ ]:
customers = pd.read_csv("customers_rating.csv", sep = ";")

customers.head(5)

,Cust_Id,Rating,Date,Movie_Id
0,1488844,3.0,2005-09-06,1
1,822109,5.0,2005-05-13,1
2,885013,4.0,2005-10-19,1
3,30878,4.0,2005-12-26,1
4,823519,3.0,2004-05-03,1


Verifica-se então se há algum dado faltante (NA) em `movies`. Isso pode ser feito aplicando os métodos `isna`, que determina em quais posições há NAs e então a contagem desses com `sum`.

In [ ]:
movies.isna().sum()

Movie_Id    0
Movie       0
Year        0
dtype: int64

Visto que não há ocorrência de informações faltantes no banco `movies`, pode-se verificar se há incidência de duas observações iguais para as colunas `Movie` e `Year`, ou seja, se um filme aparece duas vezes no _dataframe_.

Para isso, o método `loc` é chamado, passando como argumento para a filtragem o método `duplicated` que buscará duplicatas perfeitas para as colunas `Movie` e `Year`. Portanto, o resultado é um _dataframe_ que contém apenas pares perfeitos para o título e ano de um filme.

In [ ]:
movies.loc[movies.duplicated(subset = ["Movie", "Year"], keep = False), :]

,Movie_Id,Movie,Year
349,350,Dr. Quinn,1993
4004,4005,Dr. Quinn,1993


Visto que o filme "Dr. Quinn" de 1993 está duplicado no banco de dados, deve-se removê-lo. Contudo, o valor para a coluna `Movie_Id` é diferente em cada linha. Dessa forma, verifica-se se há avaliações para esse filme no banco `customers` com os dois possíveis índices.

Isso é feito então ao fazer uso da função `len`, que conta o tamanho do _dataframe_ condicionado para cada índice.

In [ ]:
print(len(customers[customers.Movie_Id == 350].index))
print(len(customers[customers.Movie_Id == 4005].index))

1151
1381


Há, então, 1151 avaliações para o índice 350 e 1381 avaliações para o índice 4005. Antes de resolver esta questão, é verificado se existem duplicatas no banco `customers`, onde um mesmo _customer_ avaliou duas vezes o mesmo filme.

In [ ]:
customers.loc[customers.duplicated(subset = ["Cust_Id", "Movie_Id"], keep = False), :]

,Cust_Id,Rating,Date,Movie_Id


Como o retorno é um _dataframe_ vazio, conclui-se que esse não é o caso. Então, será feita uma alteração para que o índice dos filmes com índice 4005 tornem-se 350.

Contudo, é fácil inferir que isso gerará duplicatas e haverá ocasiões onde alguém avaliou o índice 350 original, mas não o 4005; onde alguém avaliou somente o índice 4005; e também a situação em que alguém avaliou ambos. Para tanto, os dados serão agrupados - com apoio do método `groupby` - por `Cust_Id` e `Movie_Id` após a alteração referida e será tirada a média de `Rating`.

Para isso, faz-se uso do método `agg` que agrega as colunas utilizando as operações passadas para cada coluna. O argumento `"first"` faz com que os valores originais sejam usadas e o argumento `"mean"` faz com que a média seja tirada.

Assim, avaliações únicas terão o valor original mantido e avaliações duplas terão a média como avaliação final.

In [ ]:
customers.loc[customers["Movie_Id"] == 4005, "Movie_Id"] = 350

customers = customers.groupby(["Cust_Id", "Movie_Id"], as_index = False).agg({"Cust_Id": "first", "Rating": "mean", "Date": "first", "Movie_Id": "first"})

Após essa alteração, é feita a remoção do filme duplicado em `movies`. Além disso, houve a verificação para duplicatas em `Movie_Id`, contudo, isso foi omitido do relatório dado que estas não foram encontradas.

Para isso, é feito uso do método `drop_duplicates` no banco `movies`, que remove duplicatas de acordo com as colunas passadas no argumento `subset`.

In [ ]:
movies = movies.drop_duplicates(subset = ["Movie", "Year"])

Por fim, é verificado se há NAs no banco `customers`, da mesma forma que foi feito para o banco `movies`.

In [ ]:
customers.isna().sum()

Cust_Id     0
Rating      0
Date        0
Movie_Id    0
dtype: int64

Assim, pode-se concluir que não há duplicatas e nem NAs no banco de dados.

# 1. Análise dos bancos de dados
## 1.1 Quantos filmes estão disponíveis no dataset?

Começa-se então a análise dos bancos de dados. Primeiramente, verifica-se quantos filmes estão contidos no banco `movies`. Isso é feito através da função `len` novamente. Tem-se, então, um total de 4498 filmes no banco `movies`.

In [ ]:
print(len(movies.index))

4498


## 1.2 Qual é o nome dos 5 filmes com melhor média de avaliação?

Em seguida, é necessário um tratamento do banco de dados para a obtenção dos filmes com as melhores avaliações. Para isso, será criada uma nova coluna, `Rating`, em `movies`.

Primeiramente, o banco `rating_means` é criado, fazendo um agrupamento (com o método `groupby`) na coluna `Movie_Id` e encontrando a média dos filmes com a função `mean`. Para ser mais apresentável, os dados são então arredondados para duas casas decimais com a função `round`.

In [ ]:
rating_means = customers.groupby(["Movie_Id"], as_index = False)["Rating"].mean()
rating_means["Rating"] = rating_means["Rating"].round(2)

rating_means.head(5)

,Movie_Id,Rating
0,1,3.75
1,2,3.56
2,3,3.64
3,4,2.74
4,5,3.92


Agora, para adicionar a coluna `Rating` contida em `rating_means`, o método `join` é aplicado no banco `movies`. O método `join` junta os dois _dataframes_ com base na coluna passada no argumento `on`, que é uma coluna que ambos _dataframes_ possuem. Então, onde os valores das colunas são iguais, os _dataframes_ são agregados.

Dessa forma, as avaliações médias contidas na coluna `Rating` são agregadas corretamente aos índices dos filmes em `Movie_Id` no _dataframe_ `movies`.

In [ ]:
movies = movies.join(rating_means.set_index("Movie_Id"), on = "Movie_Id")
movies.head(5)

,Movie_Id,Movie,Year,Rating
0,1,Dinosaur Planet,2003,3.75
1,2,Isle of Man TT 2004 Review,2004,3.56
2,3,Character,1997,3.64
3,4,Paula Abdul's Get Up & Dance,1994,2.74
4,5,The Rise and Fall of ECW,2004,3.92


Agora que tem-se as avaliações médias junto com o banco `movies`, pode-se simplesmente alterar a ordenação deles para verificar quais o 5 filmes com melhores avaliações médias.

Para isso, o método `sort_values` é passado, com o argumento `by = ["Rating"]` para que a ordenação seja feita na coluna de avaliações e, também, com o argumento `ascending = False` para que a ordenação seja feita de forma decrescente, assim as primeiras observações serão aquelas com as maiores notas.

In [ ]:
movies.sort_values(by = "Rating", ascending = False).head(6)

,Movie_Id,Movie,Year,Rating
3455,3456,Lost: Season 1,2004,4.67
3032,3033,Ghost in the Shell: Stand Alone Complex: 2nd Gig,2005,4.59
2101,2102,The Simpsons: Season 6,1994,4.58
4237,4238,Inu-Yasha,2000,4.55
12,13,Lord of the Rings: The Return of the King: Ext...,2003,4.55
3443,3444,Family Guy: Freakin' Sweet Collection,2004,4.52


Portanto, os 5 "filmes" com melhores avaliações são:

1.   Lost: Season 1
2.   Ghost in the Shell: Stand Alone Complex: 2nd Gig
3.   The Simpsons: Season 6
4.   Inu-Yasha
5.   Lord of the Rings: The Return of the King: Extended Version





## Quais os 5 anos com menos lançamentos de filmes?

Busca-se agora quais os anos com menos lançamentos de filmes. Pode-se fazer uso do método `value_counts`, que conta quantas ocorrências da série (nesse caso, da coluna) indicada acontecem por valor único da série. O argumento `ascending = True` então faz com que a ordem seja crescente, para que as primeiras linhas contenham as ocorrências com menor frequência.

In [ ]:
movies["Year"].value_counts(ascending = True).head(10)

 1917    1
 1915    1
 1926    1
 1922    1
 1918    2
 1924    2
 1916    2
 1931    2
 1929    2
 1928    3
Name: Year, dtype: int64

Apresenta-se aqui os 10 anos com menor quantidade de filmes lançados pois há um empate para o quinto ano com menos lançamentos entre 1918, 1924, 1916, 1931 e 1929, todos com 2 filmes lançados. Como, também, para o ano com menos lançamentos há um empate entre 1917, 1915, 1926 e 1922 - todos com apenas 1 filme -, não é possível determinar apenas 5 anos com menos lançamentos.

Contudo, pode-se ordenar da seguinte forma:

*   Os anos 1915, 1917, 1922 e 1926 tiveram apenas um filme lançado em cada ano;
*   E os anos 1916, 1918, 1924, 1929 e 1931 tiveram dois filmes lançados em cada ano.



## 1.4 Quantos filmes que possuem avaliação maior ou igual a 4.7, considerando apenas os filmes avaliados na última data de avaliação do dataset?

Agora, é necessário verificar quais filmes possuem avaliações maiores ou iguais a 4.7, entre os filmes avaliados na última data de avaliação.

Aqui é feito uso da função `max` para a coluna `Date`, que retorna última data de avaliação.

In [ ]:
customers["Date"].max()

'2005-12-31'

A última data de avaliação é dada em 31 de dezembro de 2005. É criado, então, um novo banco que contém apenas observações dessa data para que seja mais rápido trabalhar com ele.

O novo banco `last_ratings` é então criado com auxílio do método `loc` buscando a data 2005-12-31. Após isso, os dados são agrupados por `Movie_Id` e a média das avaliações é obtida por meio da função `mean`, e então arredondada para duas casas decimais pela função `round`.

Agora que o banco `last_ratings` contém as colunas `Movie_Id`, com o índice dos filmes, e `Rating`, com a média de avaliação dos filmes, utiliza-se novamente o método `loc` para filtrar somente as avaliações médias iguais ou maiores que 4.7.

In [ ]:
last_ratings = customers.loc[customers["Date"] == "2005-12-31"]
last_ratings = last_ratings.groupby(["Movie_Id"], as_index = False)["Rating"].mean()
last_ratings["Rating"] = last_ratings["Rating"].round(2)
last_ratings = last_ratings.loc[last_ratings["Rating"] >= 4.7]

last_ratings.head(5)

,Movie_Id,Rating
2,16,5.0
12,52,5.0
13,55,5.0
26,116,5.0
28,121,5.0


Agora, pode ser encontrado o número de filmes com avaliação média igual ou maior que 4.7 para a última data de avaliações, fazendo uso novamente da função `len`.

In [ ]:
len(last_ratings)

195

Assim, tem-se 195 filmes com avaliação média iguais ou maiores que 4.7.

### 1.4.1 Outra interpretação

Da forma que a pergunta está escrita, primeiro foi entendido que o objetivo era encontrar quais filmes têm avaliação média total - isto é, considerando todas datas de avaliação - igual ou maior a 4.7, dentre os filmes que foram avaliados na última data de avaliação. Contudo, foi considerado que esse não era o objetivo da questão, visto que nenhum filme possui avaliação média igual ou maior a 4.7 considerando todas avaliações.

Esse comentário foi feito apenas por questão de minuciosidade quanto a análise e porque essa questão é base para a construção da próxima. Por conta disso, as respostas da próxima questão diferem para as interpretações desta.

Por fim, é considerado que o objetivo da questão 1.4 é tal qual o desenvolvimento acima.

## 1.5 Dos filmes encontrados na questão anterior, quais são os 10 filmes com as piores notas e quais as notas?



Deseja-se encontrar os 10 filmes com as piores notas dentre aqueles que tiveram notas maiores ou iguais a 4.7 na última data de avaliação. Primeiro, visualiza-se o _dataframe_ `last_ratings` ordenado pelas avaliações médias em ordem crescente para confirmar quais são esses 10 filmes.

Novamente, o método `sort_values` é utilizado.

In [ ]:
last_ratings.sort_values(by = "Rating").head(11)

,Movie_Id,Rating
1208,3446,4.71
1558,4409,4.75
910,2585,4.75
726,2129,4.75
664,1947,4.75
1248,3551,4.75
613,1800,4.75
419,1208,4.75
244,677,4.80
1529,4353,4.80


Agora, sabe-se que a listagem será feita realmente com 10 filmes, já que o décimo primeiro filme com pior nota não empata com o décimo. Tem-se os índices dos filmes, porém é mais apresentável representá-los por seus nomes e datas de lançamento.

Pode-se fazer isso criando, primeiro, o banco `ten_worst`, criado com as 10 piores avaliações do banco `last_ratings`, encontradas com o método `sort_values`, novamente.

Esse banco é então mesclado com o banco `movies` fazendo uso da função `merge`, que usará os argumentos `left_on` e `right_on` para determinar qual coluna usar como base para mesclar os _dataframes_, `Movie_Id` passada para os argumentos.

Então, para melhorar a visualização, a coluna `Rating_y`, que é criada na mesclagem, é removida e a coluna `Rating_x`, que é a coluna certa para as avaliações dos filmes, é renomeada para `Rating`.

Por fim, o banco com os 10 piores filmes encontrados no banco da questão 1.4 pode ser visualizado.

In [ ]:
ten_worst = last_ratings.sort_values(by = "Rating").head(10)
ten_worst = pd.merge(ten_worst, movies, left_on = "Movie_Id", right_on = "Movie_Id")
ten_worst.drop(columns = "Rating_y", inplace = True)
ten_worst.rename(columns = {"Rating_x": "Rating"}, inplace = True)

ten_worst

,Movie_Id,Rating,Movie,Year
0,3446,4.71,Spirited Away,2002
1,4409,4.75,SpongeBob SquarePants: Season 3,2002
2,2585,4.75,Absolutely Fabulous: Series 2,1994
3,2129,4.75,The Twilight Zone: Vol. 41,1960
4,1947,4.75,Gilmore Girls: Season 3,2002
5,3551,4.75,Pete's Dragon,1977
6,1800,4.75,An Evening With Kevin Smith,2002
7,1208,4.75,The Twilight Zone: Vol. 15,1963
8,677,4.80,In the Mood for Love,2001
9,4353,4.80,Curb Your Enthusiasm: Season 3,2002


## 1.6 Quais os id's dos 5 customer que mais avaliaram filmes e quantas avaliações cada um fez?

Para tanto, é criado um novo _dataframe_, `top_customers`, que contém a contagem de avaliações por _customers_.

Novamente, o método `value_counts` é usado, agora na coluna `Cust_Id` para ver quantas ocorrências de cada _customer_ estão contidas no banco `customers`. As 6 ocorrências com maior quantidade de ocorrências são então apresentadas.

In [ ]:
customers["Cust_Id"].value_counts().head(6)

305344     4466
387418     4421
2439493    4194
1664010    4018
2118461    3768
1639792    2500
Name: Cust_Id, dtype: int64

Portanto, os cinco _customers_ que mais fizeram avaliações são:

1.    Customer 305344, com 4466 avaliações;
2.    Customer 387418, com 4421 avaliações;
3.    Customer 2439493, com 4194 avaliações;
4.    Customer 1664010, com 4018 avaliações;
5.    Customer 2118461, com 3768 avaliações.